# Exemples of model builders

In [1]:
import logging
logging.basicConfig()
logger = logging.getLogger('dependencynet')
logger.setLevel(logging.INFO)

In [2]:
import sys
sys.path.append("../")  # go to parent dir to load depencencynet classes

In [3]:
from dependencynet.schema import SchemaBuilder
from dependencynet.model import ModelBuilder, ModelStorageService

In [4]:
from os import path
import pandas as pd

In [5]:
filename = path.join('..', 'tests', 'resources', 'data', 'compact', 'towns.csv')
data = pd.read_csv(filename, delimiter=';')

properties = ['area', 'country', 'town', 'monument']
source_df = pd.DataFrame(data, columns=properties)

display(source_df)

,area,country,town,monument
0,Europe,France,Paris,"Eiffel Tower,Louvre Museum"
1,Europe,France,Lyon,NaN
2,Europe,UK,London,"Tower Bridge,Tower of London"
3,Europe,Italia,Rome,Colosseum
4,Asia,Japan,Tokyo,Senso-ji


In [6]:
schema_towns = SchemaBuilder().level('area', 'A').level('country', 'C').level('town', 'T').resource('monument', 'M').render()

model = ModelBuilder().from_compact(source_df) \
                      .with_schema(schema_towns) \
                      .render()

INFO:dependencynet.schema:rendering schema
INFO:dependencynet.datasource.core.levelsloader:extract_items_root keys=['area'] id_pattern=A{id:02d} => shape=(2, 4)
INFO:dependencynet.datasource.core.levelsloader:extract_items_root keys=['area', 'country'] id_pattern={id_parent}C{id:02d} => shape=(4, 6)
INFO:dependencynet.datasource.core.levelsloader:extract_items_root keys=['area', 'country', 'town'] id_pattern={id_parent}T{id:02d} => shape=(5, 7)
INFO:dependencynet.datasource.core.resourcesloader:__extract_resource keys=['area', 'country', 'town', 'monument'] id_pattern={id_parent}M{id:02d} => shape=(4, 8)


In [7]:
dfs = model.levels_datasets

if logger.level <= logging.INFO:
    logger.info('level datasets')
    display(dfs[0])
    display(dfs[1])
    display(dfs[2])

INFO:dependencynet:level datasets


,area,pos,id,label
0,Europe,1,A01,A01 Europe
4,Asia,2,A02,A02 Asia


,area,country,pos,id_parent,id,label
0,Europe,France,1,A01,A01C01,A01C01 France
1,Europe,UK,2,A01,A01C02,A01C02 UK
2,Europe,Italia,3,A01,A01C03,A01C03 Italia
3,Asia,Japan,1,A02,A02C01,A02C01 Japan


,area,country,town,pos,id_parent,id,label
0,Europe,France,Paris,1,A01C01,A01C01T01,A01C01T01 Paris
1,Europe,France,Lyon,2,A01C01,A01C01T02,A01C01T02 Lyon
2,Europe,UK,London,1,A01C02,A01C02T01,A01C02T01 London
3,Europe,Italia,Rome,1,A01C03,A01C03T01,A01C03T01 Rome
4,Asia,Japan,Tokyo,1,A02C01,A02C01T01,A02C01T01 Tokyo


In [8]:
dfs = model.resources_datasets

if logger.level <= logging.INFO:
    logger.info('resourcez datasets')
    display(dfs['monument'])    
    display('monument' in dfs['monument'].columns)

INFO:dependencynet:resourcez datasets


,area,country,town,monument,pos,id_parent,id,label
0,Europe,France,Paris,"Eiffel Tower,Louvre Museum",1,A01C01T01,A01C01T01M01,"A01C01T01M01 Eiffel Tower,Louvre Museum"
1,Europe,UK,London,"Tower Bridge,Tower of London",1,A01C02T01,A01C02T01M01,"A01C02T01M01 Tower Bridge,Tower of London"
2,Europe,Italia,Rome,Colosseum,1,A01C03T01,A01C03T01M01,A01C03T01M01 Colosseum
3,Asia,Japan,Tokyo,Senso-ji,1,A02C01T01,A02C01T01M01,A02C01T01M01 Senso-ji


True

In [9]:
folder_name = path.join('output', 'datasets')
model.save(folder_name)

INFO:dependencynet.datasource.core.modelstorage:model folder is output/datasets/current
INFO:dependencynet.datasource.core.modelstorage:schema saved under name output/datasets/current/schema.json
INFO:dependencynet.datasource.core.modelstorage:dateset saved under name output/datasets/current/area.csv
INFO:dependencynet.datasource.core.modelstorage:dateset saved under name output/datasets/current/country.csv
INFO:dependencynet.datasource.core.modelstorage:dateset saved under name output/datasets/current/town.csv
INFO:dependencynet.datasource.core.modelstorage:dateset saved under name output/datasets/current/monument.csv
INFO:dependencynet.datasource.core.modelstorage:tree saved under name output/datasets/current/tree.json


In [10]:
display(model.tree_model)

defaultdict(<class 'dict'>, {'area_dict': {'A01': {'area': 'Europe', 'pos': 1, 'id': 'A01', 'label': 'A01 Europe', 'country_dict': {'A01C01': {'area': 'Europe', 'country': 'France', 'pos': 1, 'id_parent': 'A01', 'id': 'A01C01', 'label': 'A01C01 France', 'town_dict': {'A01C01T01': {'area': 'Europe', 'country': 'France', 'town': 'Paris', 'pos': 1, 'id_parent': 'A01C01', 'id': 'A01C01T01', 'label': 'A01C01T01 Paris', 'monument_dict': {'A01C01T01M01': {'area': 'Europe', 'country': 'France', 'town': 'Paris', 'monument': 'Eiffel Tower,Louvre Museum', 'pos': 1, 'id_parent': 'A01C01T01', 'id': 'A01C01T01M01', 'label': 'A01C01T01M01 Eiffel Tower,Louvre Museum'}}}, 'A01C01T02': {'area': 'Europe', 'country': 'France', 'town': 'Lyon', 'pos': 2, 'id_parent': 'A01C01', 'id': 'A01C01T02', 'label': 'A01C01T02 Lyon', 'monument_dict': {}}}}, 'A01C02': {'area': 'Europe', 'country': 'UK', 'pos': 2, 'id_parent': 'A01', 'id': 'A01C02', 'label': 'A01C02 UK', 'town_dict': {'A01C02T01': {'area': 'Europe', 'cou

In [11]:
lines = model.pretty_print()
display(lines)

['there are 2 area(s)',
 '  A01, A02',
 '  area A01: Europe',
 '    has 3 country(s)',
 '      A01C01, A01C02, A01C03',
 '      country A01C01: France',
 '        has 2 town(s)',
 '          A01C01T01, A01C01T02',
 '          town A01C01T01: Paris',
 '            has 1 monument(s)',
 '               A01C01T01M01',
 '                 monument A01C01T01M01: Eiffel Tower,Louvre Museum',
 '          town A01C01T02: Lyon',
 '            has 0 monument(s)',
 '      country A01C02: UK',
 '        has 1 town(s)',
 '          A01C02T01',
 '          town A01C02T01: London',
 '            has 1 monument(s)',
 '               A01C02T01M01',
 '                 monument A01C02T01M01: Tower Bridge,Tower of London',
 '      country A01C03: Italia',
 '        has 1 town(s)',
 '          A01C03T01',
 '          town A01C03T01: Rome',
 '            has 1 monument(s)',
 '               A01C03T01M01',
 '                 monument A01C03T01M01: Colosseum',
 '  area A02: Asia',
 '    has 1 country(s)',
 '   